In [1]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

In [2]:
data = pd.read_csv('trainData.data', delimiter = ' ', header = None)

In [3]:
X = data.iloc[:, :-1]
Y = data.iloc[:, -1]

In [4]:
# Standardizing data
scaler = StandardScaler()
scaler.fit(X)
scaler.transform(X)

#scaler.fit_transform(X)

array([[ 0.67429718, -0.66041367, -1.34858796, ..., -1.44298362,
         0.27526042, -1.08943129],
       [-0.11511662, -0.48380799,  0.52414012, ..., -1.36925386,
        -1.09997337,  0.70900002],
       [-2.39273398,  2.91562597,  2.1118062 , ..., -1.1587686 ,
         1.23454031,  0.19986577],
       ...,
       [-1.18934115,  0.85688533,  0.78507014, ..., -0.77075231,
         1.4621364 , -1.61840523],
       [ 0.96277293,  0.60066653, -0.00723339, ..., -0.80261814,
         1.70600332,  0.7459598 ],
       [-1.36269156,  0.96184025, -0.0798191 , ..., -1.15154195,
        -1.36968334,  1.0057203 ]])

In [5]:
X_train_raw, X_val_raw, y_train, y_val = train_test_split(X, Y, train_size=0.8, shuffle=True)

In [6]:
X_train = torch.tensor(X_train_raw.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1,1)
X_val = torch.tensor(X_val_raw.values, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32).reshape(-1, 1)

In [42]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_features, output_features):
        super().__init__()
        self.stack = nn.Sequential(
            nn.Linear(input_features, 80),
            nn.ReLU(),
            nn.Linear(80, 50),
            nn.ReLU(),
            nn.Linear(50, 10),
            nn.ReLU(),
            nn.Linear(10,output_features))
    def forward(self, x):
        logits = self.stack(x)
        return logits

#self.layer1 = nn.Linear(input_size, hidden_size)
#self.layer2 = nn.Linear(hidden_size, hidden_size)
#self.layer3 = nn.Linear(hidden_size, num_classes)
#self.activation_function = activation_function

 #   def forward(self, x):
 #       x = self.activation_function(self.layer1(x))
 #       x = self.activation_function(self.layer2(x))
  #      x = self.layer3(x)
   #     return x

In [43]:
model = NeuralNetwork(105, 1)

In [44]:
loss_fn = nn.MSELoss()
opt = optim.SGD(model.parameters(), lr=0.01)
n_epochs = 10
batch_size = 1000

In [45]:
loader = DataLoader(list(zip(X_train,y_train)), shuffle=True, batch_size=batch_size)


In [47]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
      optimizer.zero_grad()

      # Compute prediction and loss
      #print(X)
      pred = model(X)
      loss = loss_fn(pred, y)

        # Backpropagation
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      if batch % 100 == 0:
        loss, current = loss.item(), (batch + 1) * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
      for X, y in dataloader:
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(loader, model, loss_fn, opt)
    test_loop(loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.000066  [ 1000/240000]
loss: 0.000067  [101000/240000]
loss: 0.000060  [201000/240000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.000063 

Epoch 2
-------------------------------
loss: 0.000063  [ 1000/240000]
loss: 0.000062  [101000/240000]
loss: 0.000063  [201000/240000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.000062 

Epoch 3
-------------------------------
loss: 0.000059  [ 1000/240000]
loss: 0.000069  [101000/240000]
loss: 0.000057  [201000/240000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.000061 

Epoch 4
-------------------------------
loss: 0.000056  [ 1000/240000]
loss: 0.000055  [101000/240000]
loss: 0.000058  [201000/240000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.000060 

Epoch 5
-------------------------------
loss: 0.000062  [ 1000/240000]
loss: 0.000060  [101000/240000]
loss: 0.000059  [201000/240000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.000058 

Epoch 6
-------------------------------
loss: 0.000057  [ 1000/240000]